<div>
<img src=https://www.institutedata.com/wp-content/uploads/2019/10/iod_h_tp_primary_c.svg width="300">
</div>

# Lab 3.1.1 
# *Data Wrangling and Munging with Pandas*

## Part 1: Wrangling Data

The term "data wrangling" is analogous to capturing wild horses and getting them into a fenced area; the horses are data and the fencing is your computer. The more common data wrangling tasks include:

- reading flat files
- reading Excel files
- downloading from web pages
  - csv
  - html
  - json

In [3]:
import numpy as np
import pandas as pd

*It is good practice to display the library version numbers for future reference:*

In [2]:
print('Numpy: ', np.__version__)
print('Pandas: ', pd.__version__)

Numpy:  1.18.1
Pandas:  1.0.1


### CSV Files

Below are three attempts to load the file "bikeshare.csv" into a DataFrame named `bikes`. Why are they wrong?

In [3]:
# wrong:
bikes = pd.read_table('bikeshare.csv', header = None)
print(bikes.head())
print()

# wrong:
bikes = pd.read_table('bikeshare.csv', header = 1)
print(bikes.head())
print()

# wrong:
bikes = pd.read_table('bikeshare.csv', header = 0)
print(bikes.head())

                                                   0
0  instant,dteday,season,yr,mnth,hr,holiday,weekd...
1  1,2011-01-01,1,0,1,0,0,6,0,1,0.24,0.2879,0.81,...
2  2,2011-01-01,1,0,1,1,0,6,0,1,0.22,0.2727,0.8,0...
3  3,2011-01-01,1,0,1,2,0,6,0,1,0.22,0.2727,0.8,0...
4  4,2011-01-01,1,0,1,3,0,6,0,1,0.24,0.2879,0.75,...

  1,2011-01-01,1,0,1,0,0,6,0,1,0.24,0.2879,0.81,0,3,13,16
0  2,2011-01-01,1,0,1,1,0,6,0,1,0.22,0.2727,0.8,0...     
1  3,2011-01-01,1,0,1,2,0,6,0,1,0.22,0.2727,0.8,0...     
2  4,2011-01-01,1,0,1,3,0,6,0,1,0.24,0.2879,0.75,...     
3  5,2011-01-01,1,0,1,4,0,6,0,1,0.24,0.2879,0.75,...     
4  6,2011-01-01,1,0,1,5,0,6,0,2,0.24,0.2576,0.75,...     

  instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0  1,2011-01-01,1,0,1,0,0,6,0,1,0.24,0.2879,0.81,...                                                                   
1  2,2011-01-01,1,0,1,1,0,6,0,1,0.22,0.2727,0.8,0...                                        

?:
ANSWER: Case 1 treats headings as just another data row. Case 2 treats the 1st data row as the column header. Case 3 gets the header right (row 0), but reads each row as a single column (Nb. the other two make that same mistake). 

Load the file "bikeshare.csv" into a DataFrame named `bikes`, and confirm that it was loaded properly:

In [5]:
#ANSWER:
bikes = pd.read_csv('bikeshare.csv')

Note that we could have used `read.csv()` above. When is `read_table()` necessary?

?:
ANSWER: When `sep` is not the comma character, or we need fine control that `read.csv()` does not provide.

Flat files can be full of surprises. Here are some issues to watch out for:

- separator character is something other than the comma
  - ";", "|", and tab are popular
- newline character is something other than what the O/S expects 
  - Tip: Don't hard-code the character codes for carriage returns, linefeeds, etc. Use Python's built-in representation instead (e.g. Python translates "\n" to the newline character and "\t" to the tab character on any O/S).
- truncated lines
  - if there are empty fields at the end of a line it is possible that their separators will be missing, resulting in a "jagged" file
- embedded commas or quotes
  - a free-text field containing embedded commas may split into separate fields on input
  - a free-text field containing embedded quotes may not parse correctly
- unescaped characters
  - the "\" character indicates a control code to Python, which will break the I/O
    - e.g. the substring "\u0123" will be interpreted as Unicode(0123) -- which may not be what the file creator intended
  - these may need to be fixed by loading whole strings and then parsing into a new data frame
  
Tip: Most issues can be delth with by correctly specifying the parameters of the function you use to load the file. Read the doco before reading the data!

### Reading Excel Files

In [5]:
from pandas import ExcelFile  # Nb. Need to install xlrd from conda (it does not automatically install with pandas)

In [11]:
df = pd.read_excel('Iris.xls', sheet_name = 'Data')
df.head(10)


,Species_No,Petal_width,Petal_length,Sepal_width,Sepal_length,Species_name
0,1,0.2,1.4,3.5,5.1,Setosa
1,1,0.2,1.4,3.0,4.9,Setosa
2,1,0.2,1.3,3.2,4.7,Setosa
3,1,0.2,1.5,3.1,4.6,Setosa
4,1,0.2,1.4,3.6,5.0,Setosa
5,1,0.4,1.7,3.9,5.4,Setosa
6,1,0.3,1.4,3.4,4.6,Setosa
7,1,0.2,1.5,3.4,5.0,Setosa
8,1,0.2,1.4,2.9,4.4,Setosa
9,1,0.1,1.5,3.1,4.9,Setosa


So, this file appears to have an embedded table of aggregates on the same sheet as the raw data (a naughty but common practice amongst analysts).

It is usually better to load data correctly than to meddle with the source file or load it 'warts and all' and then try to parse it in code. The Pandas functions for reading files have parameters that provide the control we need. For ecxample, we could make multiple calls to `read_excel()`, using combinations of the `header`, `usecols`, `skiprows`, `nrows`, and `skipfooter` parameters to load one table at a time from a spreadsheet with multiple tables.

Load the above file without the unwanted columns:

In [22]:
#ANSWER
df = pd.read_excel('Iris.xls', sheet_name = 'Data', index_col =0)
df

,Petal_width,Petal_length,Sepal_width,Sepal_length,Species_name
Species_No,,,,,
1,0.2,1.4,3.5,5.1,Setosa
1,0.2,1.4,3.0,4.9,Setosa
1,0.2,1.3,3.2,4.7,Setosa
1,0.2,1.5,3.1,4.6,Setosa
1,0.2,1.4,3.6,5.0,Setosa
...,...,...,...,...,...
3,2.3,5.2,3.0,6.7,Verginica
3,1.9,5.0,2.5,6.3,Verginica
3,2.0,5.2,3.0,6.5,Verginica


### Importing Data Directly from the Web

We usually want to store a local copy of a data file that we download from the Web, but when data retention is not a priority it is convenient to download the data directly into our running Python environment.

#### Importing Text Files from the Web

The web is the 'wild west' of data formats. However, we can usually expect good behaviour from files that are automatically generated by a service, such as the earthquake report:

In [23]:
df = pd.read_csv('https://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/2.5_hour.csv')
df.head()

,time,latitude,longitude,depth,mag,magType,nst,gap,dmin,rms,...,updated,place,type,horizontalError,depthError,magError,magNst,status,locationSource,magSource
0,2021-01-06T02:35:44.727Z,10.4468,-62.5984,10,4.5,mb,NaN,94,1.914,0.79,...,2021-01-06T02:54:37.040Z,"35 km WSW of Güiria, Venezuela",earthquake,10.6,2,0.102,28,reviewed,us,us


#### Importing HTML Files from the Web

Working with unstructured HTML files relies heavily on library functions. This one, however, is well-structured:

In [26]:
url = 'https://www.fdic.gov/resources/resolutions/bank-failures/failed-bank-list/'
df = pd.read_html(url)
df

[                             Bank Name               City State   Cert  \
 0                    Almena State Bank             Almena    KS  15426   
 1           First City Bank of Florida  Fort Walton Beach    FL  16748   
 2                 The First State Bank      Barboursville    WV  14361   
 3                   Ericson State Bank            Ericson    NE  18265   
 4     City National Bank of New Jersey             Newark    NJ  21111   
 ..                                 ...                ...   ...    ...   
 558                 Superior Bank, FSB           Hinsdale    IL  32646   
 559                Malta National Bank              Malta    OH   6629   
 560    First Alliance Bank & Trust Co.         Manchester    NH  34264   
 561  National State Bank of Metropolis         Metropolis    IL   3815   
 562                   Bank of Honolulu           Honolulu    HI  21029   
 
                    Acquiring Institution       Closing Date  
 0                            Equit

#### Importing XML Files from the Web

XML files are semi-structured, but you're at the mercy of the file creator. If every record has the same format it will be much easier, but practical applications often require a lot of custom code. Here is an example that includes a nice parser class: http://www.austintaylor.io/lxml/python/pandas/xml/dataframe/2016/07/08/convert-xml-to-pandas-dataframe/

#### Importing JSON Files from the Web

Like XML, JSON files are semi-structured and may require work to capture the schema into a dataframe. Here is a simple example: 

In [25]:
url = 'https://raw.githubusercontent.com/chrisalbon/simulated_datasets/master/data.json'

# Load the first sheet of the JSON file into a data frame
df = pd.read_json(url, orient = 'columns')
df.head()

,integer,datetime,category
0,5,2015-01-01 00:00:00,0
1,5,2015-01-01 00:00:01,0
2,9,2015-01-01 00:00:02,0
3,6,2015-01-01 00:00:03,0
4,6,2015-01-01 00:00:04,0


## Part 2: Data Munging

Data munging is manipulating data to get it into a form that we can start running analyses on (which usually means getting the data into a DataFrame). Before we get to this stage, we may need to remove headers or footers, transpose columns to rows, split wide data tables into long ones, and so on. (Nb. Excel files can be particularly troublesome, because users can format their data in mixed, complex shapes.) Essentially, we need to follow Hadley Wickham's guidelines for tidy datasets (http://vita.had.co.nz/papers/tidy-data.html):

The end goal of the cleaning data process:

- each variable should be in one column
- each observation should comprise one row
- each type of observational unit should form one table
- include key columns for linking multiple tables
- the top row contains (sensible) variable names
- in general, save data as one file per table


### Dataset Morphology

Once we have our dataset in a DataFrame (or Series, if our data is only 1-dimensional), we can start examining its size and content.

How many rows and columns are in `bikes`?

In [28]:
#ANSWER
bikes.info
#Their are  [17379 rows x 17 columns]> in bikes

<bound method DataFrame.info of        instant      dteday  season  yr  mnth  hr  holiday  weekday  \
0            1  2011-01-01       1   0     1   0        0        6   
1            2  2011-01-01       1   0     1   1        0        6   
2            3  2011-01-01       1   0     1   2        0        6   
3            4  2011-01-01       1   0     1   3        0        6   
4            5  2011-01-01       1   0     1   4        0        6   
...        ...         ...     ...  ..   ...  ..      ...      ...   
17374    17375  2012-12-31       1   1    12  19        0        1   
17375    17376  2012-12-31       1   1    12  20        0        1   
17376    17377  2012-12-31       1   1    12  21        0        1   
17377    17378  2012-12-31       1   1    12  22        0        1   
17378    17379  2012-12-31       1   1    12  23        0        1   

       workingday  weathersit  temp   atemp   hum  windspeed  casual  \
0               0           1  0.24  0.2879  0.81     0

What are the column names in `bikes`?

In [29]:
#ANSWER
bikes
#The column names in bikes are
#instant	dteday	season	yr	mnth	hr	holiday	weekday	workingday	weathersit	temp	atemp	hum	windspeed	casual	registered	cnt


,instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,1,2011-01-01,1,0,1,0,0,6,0,1,0.24,0.2879,0.81,0.0000,3,13,16
1,2,2011-01-01,1,0,1,1,0,6,0,1,0.22,0.2727,0.80,0.0000,8,32,40
2,3,2011-01-01,1,0,1,2,0,6,0,1,0.22,0.2727,0.80,0.0000,5,27,32
3,4,2011-01-01,1,0,1,3,0,6,0,1,0.24,0.2879,0.75,0.0000,3,10,13
4,5,2011-01-01,1,0,1,4,0,6,0,1,0.24,0.2879,0.75,0.0000,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17374,17375,2012-12-31,1,1,12,19,0,1,1,2,0.26,0.2576,0.60,0.1642,11,108,119
17375,17376,2012-12-31,1,1,12,20,0,1,1,2,0.26,0.2576,0.60,0.1642,8,81,89
17376,17377,2012-12-31,1,1,12,21,0,1,1,1,0.26,0.2576,0.60,0.1642,7,83,90
17377,17378,2012-12-31,1,1,12,22,0,1,1,1,0.26,0.2727,0.56,0.1343,13,48,61


What are the data types of these columns?

In [33]:
#ANSWER
bikes.info()
#The data types of these columns and

 #0   instant     17379 non-null  int64  
 #1   dteday      17379 non-null  object 
 #2   season      17379 non-null  int64  
 #3   yr          17379 non-null  int64  
 #4   mnth        17379 non-null  int64  
 #5   hr          17379 non-null  int64  
 #6   holiday     17379 non-null  int64  
 #7   weekday     17379 non-null  int64  
 #8   workingday  17379 non-null  int64  
 #9   weathersit  17379 non-null  int64  
 #10  temp        17379 non-null  float64
 #11  atemp       17379 non-null  float64
 #12  hum         17379 non-null  float64
 #13  windspeed   17379 non-null  float64
 #14  casual      17379 non-null  int64  
 #15  registered  17379 non-null  int64  
 #16  cnt         17379 non-null  int64  
#dtypes: float64(4), int64(12), object(1)
#memory usage: 2.3+ MB

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17379 entries, 0 to 17378
Data columns (total 17 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   instant     17379 non-null  int64  
 1   dteday      17379 non-null  object 
 2   season      17379 non-null  int64  
 3   yr          17379 non-null  int64  
 4   mnth        17379 non-null  int64  
 5   hr          17379 non-null  int64  
 6   holiday     17379 non-null  int64  
 7   weekday     17379 non-null  int64  
 8   workingday  17379 non-null  int64  
 9   weathersit  17379 non-null  int64  
 10  temp        17379 non-null  float64
 11  atemp       17379 non-null  float64
 12  hum         17379 non-null  float64
 13  windspeed   17379 non-null  float64
 14  casual      17379 non-null  int64  
 15  registered  17379 non-null  int64  
 16  cnt         17379 non-null  int64  
dtypes: float64(4), int64(12), object(1)
memory usage: 2.3+ MB


What is the (row) index for this DataFrame?

In [0]:
#ANSWER
#RangeIndex: 17379 entries, 0 to 17378

https://www.dataquest.io/blog/python-json-tutorial/

## Slicing and Dicing

It is often preferable to refer to DataFrame columns by name, but there is more than one way to do this. 
Do `bikes['season']` and `bikes[['season']]` give the same object? Demonstrate:

In [35]:
#ANSWER
bikes['season']
bikes[['season']]

,season
0,1
1,1
2,1
3,1
4,1
...,...
17374,1
17375,1
17376,1
17377,1


How would we use object notation to show the first 4 rows of `atemp`?

In [69]:
#bikes.head(10)
bikes.iloc[:4,11]
#the below method also provides the same result
#bikes['atemp'].head(4)

0    0.2879
1    0.2727
2    0.2727
3    0.2879
Name: atemp, dtype: float64

In [70]:
#ANSWER
bikes['atemp'].head(4)

0    0.2879
1    0.2727
2    0.2727
3    0.2879
Name: atemp, dtype: float64

Algorithms that loop over multiple columns often access DataFrame columns by index. However, none of the following work (try them out by uncommenting / removing the "#E: " ): 

In [0]:
bikes[[0]]
E: bikes[0]
E: bikes[0,0]
E: bikes[[0,0]]

What is the correct way to access the 1st row of the DataFrame by its index?

In [76]:
#ANSWER
bikes.iloc[0]


instant                1
dteday        2011-01-01
season                 1
yr                     0
mnth                   1
hr                     0
holiday                0
weekday                6
workingday             0
weathersit             1
temp                0.24
atemp             0.2879
hum                 0.81
windspeed              0
casual                 3
registered            13
cnt                   16
Name: 0, dtype: object

What is the correct way to access the 2nd column of the DataFrame by its index?

In [80]:
#ANSWER
bikes.iloc[:,1]

0        2011-01-01
1        2011-01-01
2        2011-01-01
3        2011-01-01
4        2011-01-01
            ...    
17374    2012-12-31
17375    2012-12-31
17376    2012-12-31
17377    2012-12-31
17378    2012-12-31
Name: dteday, Length: 17379, dtype: object

## Handling Missing Values

What is the Pandas `isnull` function for? 

?
ANSWER: It is to check if their are null values in the data set or not. If their are null values we can remove or replace it by preprocessing.

We can apply `isnull` to the `bikes` DataFrame to show the result for every element:

In [81]:
bikes.isnull().head()

,instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False


However, we usually start at a higher level. How many nulls are in `bikes` altogether?

In [0]:
#ANSWER
#Their are no nulls in the bike data set because output shows false. if it showed true their are null values in the data set.

If this result were nonzero we would next want to find out which columns contained nulls. How can this be done in one line of code?

In [86]:
#ANSWER
bikes.isnull().info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17379 entries, 0 to 17378
Data columns (total 17 columns):
 #   Column      Non-Null Count  Dtype
---  ------      --------------  -----
 0   instant     17379 non-null  bool 
 1   dteday      17379 non-null  bool 
 2   season      17379 non-null  bool 
 3   yr          17379 non-null  bool 
 4   mnth        17379 non-null  bool 
 5   hr          17379 non-null  bool 
 6   holiday     17379 non-null  bool 
 7   weekday     17379 non-null  bool 
 8   workingday  17379 non-null  bool 
 9   weathersit  17379 non-null  bool 
 10  temp        17379 non-null  bool 
 11  atemp       17379 non-null  bool 
 12  hum         17379 non-null  bool 
 13  windspeed   17379 non-null  bool 
 14  casual      17379 non-null  bool 
 15  registered  17379 non-null  bool 
 16  cnt         17379 non-null  bool 
dtypes: bool(17)
memory usage: 288.6 KB


What is the Numpy object `nan` used for? (Write a descriptive answer.)

?
ANSWER: Marking a data point as invalid.

Write (and verify) a function that performs scalar division with built-in handling of the edge case (i.e. return a value instead of just trapping the error):

In [108]:
#ANSWER

# Check if given sides form a triangle or not 
# Check if given sides form a triangle or not
def scalarDivision(x,y):
    try:
        total = x/y
    except TypeError:    
        total = "You provided the wrong object type"
    return total    
        




Apply the Pandas `isna` function to the following data objects:

In [93]:
x = 2.3
y = np.nan
print(x, y)

2.3 nan


In [122]:
#ANSWER
#
pd.isna(x)
#False

pd.isna(y)
#True

True

In [118]:

array = np.array([[1, np.nan, 3], [4, 5, np.nan]])
print(array)

[[ 1. nan  3.]
 [ 4.  5. nan]]


In [124]:
#ANSWER
pd.isna(array)
#
#array([[False,  True, False],
#       [False, False,  True]])

array([[False,  True, False],
       [False, False,  True]])

How is the pandas I/O parameter `na_values` used?

The pandas I/O parameter  na_values is used to check for missing values in the data set which we need to replace to get accurate results.
Additional strings to recognize as NA/NaN. If dict passed, specific per-column NA values. See na values const below for a list of the values interpreted as NaN by default.

? ANSWER: 

## Data Profiling

### Counts

When there are categorical variables in a dataset we will want to know how many possible values there are in each column. (Nb. If the dataset is a sample of a larger one, our sample may not capture all possible values of every categorical.)

How many (different) seasons are in `bikes`?

In [7]:
#ANSWER
#from the data set their is only one season which represents the value of 1. but in categorical variables it should representing
#two seasons for example hot or cold 1 hot and 0 for cold season.
bikes['season'].value_counts()
#3    4496
#2    4409
#1    4242
#4    4232

3    4496
2    4409
1    4242
4    4232
Name: season, dtype: int64

### Ranges

Print the range of the `instant`, `dteday`, and `windspeed` columns: 

In [50]:
x = bikes['dteday'].min()
#'2011-01-01'
y = bikes['dteday'].max()
#'2012-12-31'

bikes['dteday'].to_numeric()

bikes['dteday'] = pd.to_numeric([bikes['dteday']])
#bikes['dteday'] =  bikes['dteday'].astype(int)

ValueError: invalid literal for int() with base 10: '2011-01-01'

In [172]:
#ANSWER
print("range is",(int(bikes.instant.max()) - int(bikes.instant.min())))
#range is 17378
#print("range is",(bikes.dteday.max()) - bikes.dteday.min())
#range is 17378
print("range is",bikes.windspeed.max() - bikes.windspeed.min())
#range is 0.8507

range is 17378
range is 0.8507


Compute and print the overall minimum and maximum of the numeric data columns:

In [22]:
bikes_min, bikes_max = (min(bikes.min()), max(bikes.max()))
bikes_min, bikes_max

TypeError: '<' not supported between instances of 'str' and 'int'

### Quantiles

Pandas makes computing quantiles easy. This is how to get the median of a Series:

In [187]:
bikes['atemp'].quantile(0.5)

0.4848

Of course, the `quantiles` method can take a tuple as its argument. Compute the 10th, 25th, 50th, 75th, and 90th percentiles in one line of code: 

In [190]:
#ANSWER
bikes['atemp'].quantile([0.10,0.25,0.5,0.75,0.90])

#0.10    0.2424
#0.25    0.3333
#0.50    0.4848
#0.75    0.6212
#0.90    0.6970
#Name: atemp, dtype: float64

0.10    0.2424
0.25    0.3333
0.50    0.4848
0.75    0.6212
0.90    0.6970
Name: atemp, dtype: float64

### Cuts

Sometimes we want to split the sample not by the quantiles of the distribution but by the range of the data. Let's take a closer look at `atemp`:

In [191]:
type(bikes['atemp'])

pandas.core.series.Series

In [192]:
bikes.sample(5)

,instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
1759,1760,2011-03-19,1,0,3,0,0,6,0,2,0.60,0.6212,0.53,0.2537,26,50,76
14242,14243,2012-08-21,3,1,8,15,0,2,1,1,0.76,0.6667,0.37,0.0000,76,226,302
16960,16961,2012-12-14,4,1,12,11,0,5,1,1,0.32,0.3030,0.65,0.1940,31,180,211
4931,4932,2011-07-29,3,0,7,9,0,5,1,1,0.86,0.8030,0.47,0.2537,28,151,179
9932,9933,2012-02-23,1,1,2,21,0,4,1,1,0.44,0.4394,0.51,0.2239,17,205,222


Suppose we decide to sort these values into 4 bins of equal width, but we want to apply the resulting groups to the entire DataFrame. Basically, we need to add a row label that indcates which bin each sample belongs in. Let's call this label "atemp_level", and use the `cut` method to populate it:

In [193]:
atemp_level = pd.cut(bikes['atemp'], bins = 4)         

What is `atemp_level`?

In [194]:
#ANSWER
atemp_level
#Atemp_level is 
#0        (0.25, 0.5]
#1        (0.25, 0.5]
#2        (0.25, 0.5]
#3        (0.25, 0.5]
#4        (0.25, 0.5]
            ...     
#17374    (0.25, 0.5]
#17375    (0.25, 0.5]
#17376    (0.25, 0.5]
#17377    (0.25, 0.5]
#17378    (0.25, 0.5]
#Name: atemp, Length: 17379, dtype: category
#Categories (4, interval[float64]): [(-0.001, 0.25] < (0.25, 0.5] < (0.5, 0.75] < (0.75, 1.0]]

0        (0.25, 0.5]
1        (0.25, 0.5]
2        (0.25, 0.5]
3        (0.25, 0.5]
4        (0.25, 0.5]
            ...     
17374    (0.25, 0.5]
17375    (0.25, 0.5]
17376    (0.25, 0.5]
17377    (0.25, 0.5]
17378    (0.25, 0.5]
Name: atemp, Length: 17379, dtype: category
Categories (4, interval[float64]): [(-0.001, 0.25] < (0.25, 0.5] < (0.5, 0.75] < (0.75, 1.0]]

Here is a random sample of `atemp_level`:

In [195]:
atemp_level.sample(5)          

7674     (0.25, 0.5]
8046     (0.25, 0.5]
14887    (0.5, 0.75]
10583    (0.25, 0.5]
15482    (0.25, 0.5]
Name: atemp, dtype: category
Categories (4, interval[float64]): [(-0.001, 0.25] < (0.25, 0.5] < (0.5, 0.75] < (0.75, 1.0]]

So, by default, `cut` produces labels that indicate the bin boundaries for each element in the series it was applied to. Usually, we will specify labels that are appropriate to the discretisation we are applying:

In [196]:
atemp_level = pd.cut(bikes['atemp'], bins = 4, labels = ["cool", "mild", "warm", "hot"])
atemp_level.sample(5)          

13831     hot
173      cool
13312    warm
15197    warm
7359     mild
Name: atemp, dtype: category
Categories (4, object): [cool < mild < warm < hot]

Incorporate the new `atemp_level` column into the `bikes` DataFrame and use it to count the number of "mild" `atemp` entries in `season` 2:

In [204]:
#ANSWER
bikes['atemp_level'] = atemp_level

,instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt,atemp_level
0,1,2011-01-01,1,0,1,0,0,6,0,1,0.24,0.2879,0.81,0.0000,3,13,16,mild
1,2,2011-01-01,1,0,1,1,0,6,0,1,0.22,0.2727,0.80,0.0000,8,32,40,mild
2,3,2011-01-01,1,0,1,2,0,6,0,1,0.22,0.2727,0.80,0.0000,5,27,32,mild
3,4,2011-01-01,1,0,1,3,0,6,0,1,0.24,0.2879,0.75,0.0000,3,10,13,mild
4,5,2011-01-01,1,0,1,4,0,6,0,1,0.24,0.2879,0.75,0.0000,0,1,1,mild
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,96,2011-01-05,1,0,1,4,0,3,1,1,0.24,0.2273,0.48,0.2239,0,2,2,cool
96,97,2011-01-05,1,0,1,5,0,3,1,1,0.22,0.2273,0.47,0.1642,0,3,3,cool
97,98,2011-01-05,1,0,1,6,0,3,1,1,0.20,0.1970,0.47,0.2239,0,33,33,cool
98,99,2011-01-05,1,0,1,7,0,3,1,1,0.18,0.1818,0.43,0.1940,1,87,88,cool


*Nb. The `atemp_level` variable we created is what the R language calls a "factor". Pandas has introduced a new data type called "category" that is similar to R's factors.*

# Synthetic Data

Sometimes we may want to generate test data, or we may need to initalise a series, matrix, or data frame for input to an algorithm. Numpy has several methods we can use for this.

Execute the following, then check the shape and content of each variable:

In [205]:
# Creating arrays with initial values
a = np.zeros((3))
b = np.ones((1,3))
c = np.random.randint(1,10,(2,3,4))   # randint(low, high, size)
d = np.arange(4)
e = np.array( [[1,2,3,4], [5,6,7,8]] )

In [215]:
# Cleaning Data
a
#array([0., 0., 0.])
b
#array([[1., 1., 1.]])
c
#array([[[4, 9, 1, 1],
#        [4, 9, 1, 9],
#        [4, 8, 2, 8]],

#       [[6, 5, 8, 3],
#        [1, 7, 2, 4],
#       [9, 6, 8, 2]]])

d
#array([0, 1, 2, 3])
e
#array([[1, 2, 3, 4],
#       [5, 6, 7, 8]])


a.shape
#(3,)
b.shape
#(1, 3)
c.shape
#(2, 3, 4)
d.shape
#(4,)
e.shape
#(2, 4)

(2, 4)

## Load Data

Load rock.csv and clean the dataset.



In [24]:
rock  = pd.read_csv("rock.csv")
rock.head(10)

,Song Clean,ARTIST CLEAN,Release Year,COMBINED,First?,Year?,PlayCount,F*G
0,Caught Up in You,.38 Special,1982,Caught Up in You by .38 Special,1,1,82,82
1,Fantasy Girl,.38 Special,NaN,Fantasy Girl by .38 Special,1,0,3,0
2,Hold On Loosely,.38 Special,1981,Hold On Loosely by .38 Special,1,1,85,85
3,Rockin' Into the Night,.38 Special,1980,Rockin' Into the Night by .38 Special,1,1,18,18
4,Art For Arts Sake,10cc,1975,Art For Arts Sake by 10cc,1,1,1,1
5,Kryptonite,3 Doors Down,2000,Kryptonite by 3 Doors Down,1,1,13,13
6,Loser,3 Doors Down,2000,Loser by 3 Doors Down,1,1,1,1
7,When I'm Gone,3 Doors Down,2002,When I'm Gone by 3 Doors Down,1,1,6,6
8,What's Up?,4 Non Blondes,1992,What's Up? by 4 Non Blondes,1,1,3,3
9,Take On Me,a-ha,1985,Take On Me by a-ha,1,1,1,1


## Check Column Names

Check column names and clean.



In [221]:
rock.isnull()

,Song Clean,ARTIST CLEAN,Release Year,COMBINED,First?,Year?,PlayCount,F*G
0,False,False,False,False,False,False,False,False
1,False,False,True,False,False,False,False,False
2,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...
2225,False,False,True,False,False,False,False,False
2226,False,False,False,False,False,False,False,False
2227,False,False,False,False,False,False,False,False
2228,False,False,False,False,False,False,False,False


# Replace Null Values With 0

Check 'release' column whether this column have any null value or not. Replace null value with 0.

In [232]:
rock
#rf=rf.replace(to_replace=" NULL",value=0)
rock['Release Year'] = rock['Release Year'].fillna(0)

rock

,Song Clean,ARTIST CLEAN,Release Year,COMBINED,First?,Year?,PlayCount,F*G
0,Caught Up in You,.38 Special,1982,Caught Up in You by .38 Special,1,1,82,82
1,Fantasy Girl,.38 Special,0,Fantasy Girl by .38 Special,1,0,3,0
2,Hold On Loosely,.38 Special,1981,Hold On Loosely by .38 Special,1,1,85,85
3,Rockin' Into the Night,.38 Special,1980,Rockin' Into the Night by .38 Special,1,1,18,18
4,Art For Arts Sake,10cc,1975,Art For Arts Sake by 10cc,1,1,1,1
...,...,...,...,...,...,...,...,...
2225,She Loves My Automobile,ZZ Top,0,She Loves My Automobile by ZZ Top,1,0,1,0
2226,Tube Snake Boogie,ZZ Top,1981,Tube Snake Boogie by ZZ Top,1,1,32,32
2227,Tush,ZZ Top,1975,Tush by ZZ Top,1,1,109,109
2228,TV Dinners,ZZ Top,1983,TV Dinners by ZZ Top,1,1,1,1


# Check Datatypes of Dataset

Check datatypes of the dataset. Is there any column which should be int instead of object? Fix the column. 


In [30]:


rock.dtypes

#rock['Release Year'] = pd.to_numeric(rock['Release Year'])


Song Clean      object
ARTIST CLEAN    object
Release Year    object
COMBINED        object
First?           int64
Year?            int64
PlayCount        int64
F*G              int64
dtype: object

# Check Min, Max of Each Column

Is there any illogical value in any column? How can we fix that?

In [34]:
rock.min()
rock.max()

Song Clean                     Ziggy Stardust
ARTIST CLEAN                             a-ha
COMBINED        Ziggy Stardust by David Bowie
First?                                      1
Year?                                       1
PlayCount                                 142
F*G                                       142
dtype: object

In [35]:
rock.isnull()
#There is ilogical value in Release Year
rock['Release Year']= rock['Release Year'].fillna(0,inplace=True)

# Write Some Functions

rock["First"]

In [44]:
prirock['F*G'].mean()
rock['PlayCount'].mean()
rock['F*G'].sum()
rock['PlayCount'].sum()
rock['F*G'].count()
rock['PlayCount'].count()

2230

## Write a function that will take a row of a DataFrame and print out the song, artist, and whether or not the release date is < 1970

In [ ]:
def release_info(row):
        s = rocksfile['Song_Clean'] 
     ' was released by ' 
     rocksfile['Artist_Clean'] 
     pd.Series(np.where(rocksfile['Release_Year'] < 1970, 'before', 'after'))
     ' 1970'

rocksfile['new'] = s

## Write a function that converts cells in a DataFrame to float and otherwise replaces them with np.nan

In [ ]:
rock['Release Year'] = rock['Release Year'].astype(float)


## Apply these functions to your dataset

In [48]:
rock['Release Year'] = rock['Release Year'].astype(float)


rock.info()

#
#Successfully converted release year to float

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2230 entries, 0 to 2229
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Song Clean    2230 non-null   object 
 1   ARTIST CLEAN  2230 non-null   object 
 2   Release Year  0 non-null      float64
 3   COMBINED      2230 non-null   object 
 4   First?        2230 non-null   int64  
 5   Year?         2230 non-null   int64  
 6   PlayCount     2230 non-null   int64  
 7   F*G           2230 non-null   int64  
dtypes: float64(1), int64(4), object(3)
memory usage: 139.5+ KB


## Describe the new float-only DataFrame.
It means that it converts the datype to floats.



> 
>
>




>


>


>




---



---



> > > > > > > > > © 2019 Institute of Data


---



---



